In [1]:
# install pycomm3 and ipywidgets packages
!pip install -q pycomm3 ipywidgets

In [2]:
import pycomm3

# Discover the devices on the network
pycomm3.CIPDriver.discover()
# Create a new CIPDriver object
pump_ip = "192.168.1.25"

# some test code to read and write tags
with pycomm3.LogixDriver(pump_ip) as conn:
    # Reading the "Status OK" parameter
    status_ok = conn.read_tag('Status_OK')
    print(f"Status OK: {status_ok.value}")

    # Writing to the "Pump Running" parameter
    conn.write_tag('Pump_Running', 1)

CommError: failed to open a connection

In [3]:
import pycomm3
from pycomm3 import Tag

class PumpController:
    """
    A PumpController class to handle communication with a pump using EtherNet/IP 
    protocol. It allows reading and writing parameters to the pump.
    """
    def __init__(self, ip_address):
        """
        Initializes the PumpController with the given IP address of the pump.

        Parameters:
        ip_address (str): The IP address of the pump.
        """
        self.ip_address = ip_address

        #! potential error here, don't how the tag list is initialized automatically or not
        #! or maybe the pump don't work with this package at all, then we have to fall back to the CIPDriver
        with pycomm3.LogixDriver(ip_address, init_tags=True) as pump:
            self.pump = pump
        #! potential error here, don't how the tag list is initialized automatically or not

        # I don't if open() is necessary here, official doc doesn't mention it
        open_return = self.pump.open()
        print(f"Open return: {open_return}")

        # Check if the connection was successful
        if self.pump.connected:
            print(f"Connected to pump at {ip_address}")

            # Print PLC info
            plc_info = self.pump.get_plc_info()
            # collect all tags definitions
            self.collect_and_print_tags_definitions()

            print("PLC Info:")
            for key, value in plc_info.items():
                print(f"{key}: {value}")
        else:
            print(f"Failed to connect to pump at {ip_address}")

    def collect_and_print_tags_definitions(self):
        """
        Collects all tag definitions from the pump and prints each tag and its attributes.

        Returns:
        None
        """
        try:
            # Get the tag list from the pump
            tag_list = self.pump.get_tag_list()
            # Print each tag and its attributes
            for tag_info in tag_list:

                #! I expect failure here, unsure how tag in pycomm3.tag.Tag is work
                tag = pycomm3.tag.Tag(
                    tag=tag_info['tag_name'],
                    value=None,
                    type=tag_info.get('data_type'),
                    error=None
                )
                print(f"Tag: {tag.tag}")
                print(f"Type: {tag.type}")
                print(f"Array Length: {tag_info.get('array_length', 'N/A')}")
                print(f"External Access: {tag_info.get('external_access', 'N/A')}")
                print(f"Structure: {tag_info.get('structure', 'N/A')}")
                print("---------------------------")
                #! I expect failure here, unsure how tag in pycomm3.tag.Tag is work

        except Exception as e:
            print(f"Error collecting tag list: {e}")

    def read_param(self, param_name):
        """
        Reads the value of the specified parameter from the pump.

        Parameters:
        param_name (str): The name of the parameter to read.

        Returns:
        A pycomm3.tag.Tag object containing the value of the parameter.
        From https://docs.pycomm3.dev/en/latest/getting_started.html#response-tag-object
        A pycomm3.tag.Tag object will be returned regardless of the success of the read operation.
        The success of the read operation can be determined by checking the error attribute of the Tag object is None or not.
        """
        try:
            return self.pump.read(param_name)
        # if we go here, we encounter unknown error, 
        except Exception as e:
            print(f"Error reading {param_name}: {e}, unknown error")
            # construct a pycomm3.Tag with error message
            # return the error message as string
            return pycomm3.tag.Tag(tag=param_name, value=None, type=None, error=str(e))
        
    def write_param(self, param_name, value):
        """
        Writes a value to the specified parameter on the pump.

        Parameters:
        param_name (str): The name of the parameter to write to.
        value: The value to write to the parameter.

        Returns:
        A pycomm3.Tag object indicating the result of the write operation.
        """
        try:
            return self.pump.write((param_name, value))
        except Exception as e:
            print(f"Error writing {value} to {param_name}: {e}")
            # Construct a pycomm3.Tag with error message
            return pycomm3.tag.Tag(tag=param_name, value=value, type=None, error=str(e))

    # this is for testing purpose, check if the tag list is the same as the tags property in the LogixDriver
    # Code copy from the official doc
    # https://docs.pycomm3.dev/en/latest/examples/tag_examples.html#tag-list
    def tag_list_equal(self):
        """
        Checks if the tag list retrieved matches the tags property in the LogixDriver.

        Returns:
        None
        """
        try:
            with pycomm3.LogixDriver(self.ip_address) as plc:
                tag_list = plc.get_tag_list()
                if {tag['tag_name']: tag for tag in tag_list} == plc.tags:
                    print('They are the same!')

            with pycomm3.LogixDriver(self.ip_address, init_tags=False) as plc2:
                plc2.get_tag_list()

            if plc.tags == plc2.tags:
                print('Calling get_tag_list() does the same thing.')
            else:
                print('Calling get_tag_list() does NOT do the same.')
        except Exception as e:
            print(f"Error in tag_list_equal: {e}")

In [4]:
import ipywidgets as widgets
from IPython.display import display

def discover_and_select_device():
    # Discover devices on the network
    devices = pycomm3.CIPDriver.discover()
    
    device_options = [f"{device['address']} - {device['product_name']}" for device in devices]
    
    # Create selection dropdown and button
    device_selector = widgets.Dropdown(options=[""] + device_options, description="Select Device:")
    submit_button = widgets.Button(description="Submit")
    exit_button = widgets.Button(description="Exit")
    output = widgets.Output(layout={'border': '1px solid black'})
    
    # Arrange widgets in rows
    row1 = widgets.HBox([device_selector])
    row2 = widgets.HBox([submit_button, exit_button])
    row3 = widgets.HBox([output])
    layout = widgets.VBox([row1, row2, row3], layout=widgets.Layout(align_items='center'))
    
    def on_submit_button_clicked(b):
        with output:
            output.clear_output()
            if not device_selector.value:
                print("Please select a device.")
            else:
                selected_device = device_selector.value.split(" - ")[0]
                start_interactive_ui(selected_device)
                output.clear_output()  # Clear output to avoid double printing

    def on_exit_button_clicked(b):
        with output:
            output.clear_output()
            print("Exiting...")
    
    submit_button.on_click(on_submit_button_clicked)
    exit_button.on_click(on_exit_button_clicked)
    
    display(layout)

def start_interactive_ui(ip_address):
    controller = PumpController(ip_address)
    
    def read_param_widget(param_name):
        result = controller.read_param(param_name)
        print(f"Read Result: {result}")

    def write_param_widget(param_name, value):
        result = controller.write_param(param_name, value)
        print(f"Write Result: {result}")

    read_param_name = widgets.Text(description="Read Param Name:")
    read_param_button = widgets.Button(description="Read Parameter")
    read_param_output = widgets.Output()

    write_param_name = widgets.Text(description="Write Param Name:")
    write_param_value = widgets.Text(description="Write Param Value:")
    write_param_button = widgets.Button(description="Write Parameter")
    write_param_output = widgets.Output()

    exit_button = widgets.Button(description="Exit")

    def on_read_button_clicked(b):
        with read_param_output:
            read_param_output.clear_output()
            read_param_widget(read_param_name.value)

    def on_write_button_clicked(b):
        with write_param_output:
            write_param_output.clear_output()
            write_param_widget(write_param_name.value, write_param_value.value)
    
    def on_exit_button_clicked(b):
        with read_param_output, write_param_output:
            read_param_output.clear_output()
            write_param_output.clear_output()
            print("Exiting interactive UI...")

    read_param_button.on_click(on_read_button_clicked)
    write_param_button.on_click(on_write_button_clicked)
    exit_button.on_click(on_exit_button_clicked)

    display(read_param_name, read_param_button, read_param_output)
    display(write_param_name, write_param_value, write_param_button, write_param_output)
    display(exit_button)

# Start the device discovery and selection process
discover_and_select_device()